<a href="https://colab.research.google.com/github/abbyambita/Backpropagation-Algorithm-Implementation/blob/master/resnet_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Google Authentication

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
import os

os.chdir("/content/gdrive/My Drive")

!ls  '/content/gdrive/My Drive/CS 300/Code/'

%cd "/content/gdrive/My Drive/CS 300/Code"

acgan_output_images	 renamed_labeled_dataset_12x12
build_datasets.ipynb	 resnet_regression_12x12.ipynb
dataset			 resnet_regression.ipynb
image2mass		 transfer_learning_models_1212.ipynb
labeled_dataset		 transfer_learning_models.ipynb
Mass_Flow_Estimation	 transfer_learning_with_acgan_kaggle.ipynb
model_backup		 trial_acgan_kaggle.ipynb
model_result		 tsne_acgan_git.ipynb
output_result		 Untitled0.ipynb
plots			 ViT_H_14.ipynb
renamed_labeled_dataset  ViT-pytorch
/content/gdrive/.shortcut-targets-by-id/15tH5Hjusd1sYm2KqoKTIP43Ew95os4Xp/CS 300/Code


## Load the libraries

In [3]:
import argparse
import torch
import os
from torchvision import transforms, utils, datasets, models
from torch.utils.data import DataLoader, ConcatDataset, Dataset
from torch import nn
from torch import optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
from collections import OrderedDict
import time
from PIL import Image
import pandas as pd
import numpy as np
import glob
import copy
import shutil
import sys
from google.colab import files
import matplotlib.pyplot as plt

In [4]:
dir = r'labeled_dataset'

def remove_stray_files(image_path):
  """
  Remove the labeled-dataset directory everytime the script is run
  to avoid the chance of stray images being present in the directory
  :param image_path - a directory specifying the source of images
  return - None
  """
  try:
    shutil.rmtree(image_path)
  except OSError:
    print("directory  doesn't exist")

In [5]:
def parse_gauze_info(path, file):
  """
  Parse the Gauze info file that
  contains the numerical data for
  each gauze
  :param path - a directory to the file
  :param file - a text file delimited by space
  return - a DataFrame
  """
  gauzes = pd.read_csv(os.path.join(path, file))
  gauzes['amount(ml)'] = gauzes['amount(ml)'].astype('int')

  
  return gauzes[['filename','amount(ml)','gauze_id']]

In [ ]:
#rename the images in labeled_dataset by the id 

# gauzes = parse_gauze_info(r'labeled_dataset', 'labels_4x4.csv')

# for id in gauzes['filename']:
#   gauze_id = gauzes[gauzes['filename']==id]['gauze_id'].values[0]
#   os.rename('labeled_dataset/'+id, 'renamed_labeled_dataset/'+ str(gauze_id) + ".png")


In [6]:
from sklearn.model_selection import KFold
def split_into_train_test_valid(image_path, split_ratio=0.1):
  gauze_ids = set()
  for file in os.listdir(image_path):
    if file.endswith('.png'):
      gauze_ids.add(int(file.split('.')[0]))

  # gauze_ids = list(gauze_ids)  
  # np.random.shuffle(gauze_ids)    
  # num_gauzes = len(gauze_ids)
  # split_num = int(num_gauzes * split_ratio)
  # test_ids = gauze_ids[:split_num]
  # train_ids = gauze_ids[split_num:]
  # num_gauzes = len(train_ids)
  # split_num = int(num_gauzes * split_ratio)
  # valid_ids = train_ids[:split_num]
  # train_ids = train_ids[split_num:]

  kf = KFold(n_splits=10)
  gauze_ids = np.array(list(gauze_ids))
  num_gauzes = len(gauze_ids)

  train_ids = []
  test_ids = []
  valid_ids = []

  split_num = int(num_gauzes * split_ratio)

  for i, (train_index, test_index) in enumerate(kf.split(gauze_ids)):
    train_id = gauze_ids[train_index]
    test_id = gauze_ids[test_index]

    split_num = int(num_gauzes * split_ratio)
    val_id = train_id[:split_num]
    train_id = train_id[split_num:]

    test_ids.append(test_id)
    valid_ids.append(val_id)
    train_ids.append(train_id)


  return train_ids, test_ids, valid_ids

In [7]:
def make_dirs_copy_images(image_path, split, *gauze_ids):
  """
  Make train, test and validation directories
  and copy the files from `image_path` to their
  respective directories using `gauze_ids`
  :param image_path - a directory specifying the source of images
  :param gauze_ids - a list of gauze ids for train, test and validation images
  return - None
  """
  try:
    os.makedirs(os.path.join(image_path+"/"+split, 'train'), exist_ok=True)
  except FileExistsError:
    pass
  
  try:
    os.makedirs(os.path.join(image_path+"/"+split, 'test'), exist_ok=True)
  except FileExistsError:
    pass
  
  try:
    os.makedirs(os.path.join(image_path+"/"+split, 'valid'), exist_ok=True)
  except FileExistsError:
    pass
  
  train_ids, test_ids, valid_ids = gauze_ids
  for file in os.listdir(image_path):
    gauze_id = file.split('.')[0]
    if file.endswith('.png'):
      if int(gauze_id) in train_ids:
        shutil.copy(os.path.join(image_path, file), 
                   os.path.join(image_path+"/"+split, 'train', file))
      elif int(gauze_id) in test_ids:
        shutil.copy(os.path.join(image_path, file), 
                   os.path.join(image_path+"/"+split, 'test', file))
      else:
        shutil.copy(os.path.join(image_path, file), 
                   os.path.join(image_path+"/"+split, 'valid', file))

In [8]:
def load_images(gauze_ids, image_path, folder):
  """
  Load the images for a given gauze in sorted
  order so it is uniform for all gauzes
  :param gauze_ids - a list of gauze ids
  :param image_path - a path specifying the directory to the source images
  :param folder - a folder, whether train, test or valid
  return - a sorted list of gauze image paths
  """
  gauzes_paths = []
  for idx in gauze_ids:
    # bedroom, bathroom, frontal view and kitchen images
    source_image_path = os.path.join(image_path, folder, "{}.png".format(idx))
    files = list(glob.glob(source_image_path))
    gauzes_paths.append(sorted(files))

  #print(gauzes_paths)
  return gauzes_paths

In [9]:
def tile_images(files_list, size=128):
  print(files_list)

  gauzes = []
  #tile_size = (size, size)
  for gauze in files_list:
    
    image = Image.open(gauze)
    image = image.convert('RGB')
    # img_list = []
    # tiled_image = np.zeros((tile_size[0] * 2, tile_size[1] * 2, 3), dtype='uint8')
    # for i, location in enumerate(house):
    #   image = Image.open(location)
    #   image = image.convert('RGB')
    #   img_list.append(image.resize(tile_size))
    # tiled_image = np.vstack([np.hstack([img_list[0], img_list[1]]), 
    #                          np.hstack([img_list[2], img_list[3]])])
    # convert to PIL from numpy
    gauzes.append(Image.fromarray(image))
  return gauzes

In [10]:
class GauzeDataset(Dataset):
  """
  Custom dataset class for gauzes dataset
  Load all gauze images
  Index through each tiled gauze and its amount
  """
  
  def __init__(self, df, gauze_ids, path, folder, size, transform=None):
    self.path = path
    self.files = load_images(gauze_ids, path, folder)
    self.transform = transform
    self.df = df
    self.gauze_ids = gauze_ids
    #self.size = size
    # self.gauzes = tile_images(self.files, self.size)
    self.scale_factor = 1e6

  def __len__(self):
    return len(self.files)
  
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
            idx = idx.tolist()

    # given that the response variable is the gauze price
    # we will pass that along with the respective gauze
    # Log transform the amount
    # if self.transform:
    #   #return self.transform(self.gauzes[idx]), torch.tensor(np.log(self.df.loc[self.gauze_ids[idx]-1, 'amount']))
    #   return self.transform(self.files[idx]), torch.tensor(np.log(self.df.loc[self.gauze_ids[idx]-1, 'amount(ml)']))

    # return self.files[idx], torch.tensor(np.log(self.df.loc[self.gauze_ids[idx]-1, 'amount(ml)']))

    #img_path = self.files[idx][0]
    img_path = self.files[idx][0]
    image = Image.open(img_path).convert('RGB')

    if self.transform:
        image = self.transform(image)

    #sample = {image, np.log(self.df.loc[self.gauze_ids[idx]-1, 'amount(ml)'])}
    #return image, torch.tensor(np.log(self.df.loc[self.gauze_ids[idx]-1, 'amount(ml)']))
    return image, torch.tensor(self.df.loc[self.gauze_ids[idx]-1, 'amount(ml)'])

In [11]:
#load_data(image_dir, data_type, df, gauze_ids, img_size, image_transforms[data_type])
def load_data(data_dir, data, df, gauze_ids, size=128, image_transform=None):
    """
    Load the train, test and validation datasets from their respective
    directories
    :param data_dir - a string specifying the base directory
    :param data - a string specifying the type of dataset
    :param df - a DataFrame containing the housing information
    :param gauze_ids - a list of gauze ids
    :param size - an integer specifying the size of the image
    :param image_transforms - a transform object for that data type
    return - a dictionary mapping the data type to the location of the image dataset
    """
    return {data: GauzeDataset(df, gauze_ids, data_dir, data, size, transform=image_transform)}


def transform_images(size=224):
  """
  Apply transformations separately to the train, test and validation
  datasets
  :param size - an integer specifying the size of the image
  return - a dictionary mapping the type of dataset to its transformaions
  """
  data_transforms = {
                    'train': transforms.Compose([
                                                  # transforms.ColorJitter(
                                                  #                       brightness=0.2*torch.rand(1).item(), 
                                                  #                       hue=0.1*torch.rand(1).item(), 
                                                  #                       saturation=0.3*torch.rand(1).item(),
                                                  #                       contrast=0.3*torch.rand(1).item()
                                                  #                       ),
                                                  transforms.CenterCrop(size),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(
                                                                   [0.485, 0.456, 0.406],
                                                                   [0.229, 0.224, 0.225]
                                                                      )
                                                ]),
                      'valid': transforms.Compose([
                                                  transforms.CenterCrop(size),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(
                                                                   [0.485, 0.456, 0.406],
                                                                   [0.229, 0.224, 0.225]
                                                                      )
                                                 ]),
                      'test': transforms.Compose([
                                                  transforms.CenterCrop(size),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(
                                                                   [0.485, 0.456, 0.406],
                                                                   [0.229, 0.224, 0.225]
                                                                    )
                                                 ])
  }
  return data_transforms
  
def data_loader(image_datasets, data, size, shuffle=False):
    """
    Create generator objects from the image datasets
    :param image_datasets - an ImageFolder dataset object for that data type
    :param data - a string specifying the dataset type
    :param size - an integer specifying the size of the batch
    :param shuffle - a bool
    return a dictionary mapping the data type to the dataloader object
    """
    return {data: torch.utils.data.DataLoader(image_datasets, batch_size=size, shuffle=shuffle)}

## Implement the Classifier

In [12]:
def build_model(arch, num_classes, hidden_units=1024):
  """
  Load a pretrained model with only the final layer
  replaced by a user-defined classifier
  :param arch - a string specifying the type of model architecture
  :param num_classes - an integer specifying the number of class labels
  :param hidden_units - an integer specifying the size
  return - a pretrained model with a user-defined classifier
  """
  in_features = 0
  try:
        # model = eval("models." + arch + "(pretrained=True)")
        model = models.__dict__[arch](pretrained=True)
  except:
      raise Exception('Invalid architecture specified')
    
  # Freeze parameters as only the final layer is being trained
  for param in model.parameters():
      param.require_grad = False
  # extract the last layer in the model
  last_layer = list(model.children())[-1]
  if isinstance(last_layer, nn.Sequential):
      count = 0
      for layer in last_layer:
          if isinstance(layer, nn.Linear):
              # fetch the first of the many Linear layers
              count += 1
              in_features = layer.in_features
          if count == 1:
              break
  elif isinstance(last_layer, nn.Linear):
      in_features = last_layer.in_features
  # define the new classifier
  classifier = nn.Sequential(OrderedDict([
                          ('bc1', nn.BatchNorm1d(in_features)),
                          ('relu1', nn.ReLU()),
                          ('fc1', nn.Linear(in_features, num_classes, bias=True)),
  ]))
  # replace the existing classifier in thelast layer with the new one
  if model.__dict__['_modules'].get('fc', None):
      model.fc = classifier
  else:
      model.classifier = classifier

  return model

In [13]:
def train_model(model, trainloader, validloader, criterion, optimizer, scheduler, epochs, diff_lr=False, device='cpu'):
  
  """
  Train the model and run inference on the validation dataset. Capture the loss
  of the trained model and validation model. Also display the accuracy of the
  validation model
  :param model - a pretrain model object
  :param trainloader - a generator object representing the train dataset
  :param validloader - a generator object representing the validation dataset
  :param criterion - a loss object
  :param optimizer - an optimizer object
  :param scheduler - a scheduler object that varies the learning rate every n epochs
  :param epochs - an integer specifying the number of epochs to train the model
  :param diff_lr - a boolean specifying whether to use differential learning rate
  :param device - a string specifying whether to use cuda or cpu
  return a trained model with the best weights
  """
  start = time.time()
  print_every = 50
  steps = 0
  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0
  valid_loss_min = np.Inf
  training_loss, validation_loss = [], []
  for epoch in range(epochs):
      lr_used = 0
      if diff_lr:
        for param in optimizer.param_groups:
          if param['lr'] > lr_used:
            lr_used = param['lr']
        print('learning rate being used {}'.format(lr_used))      
      running_loss = 0
      # train_acc = 0
      scheduler.step()
      model.train()
      for idx, (images, prices) in enumerate(trainloader):
          steps += 1

          images, prices = images.to(device), prices.to(device)


        #added for MSE
          prices = prices.to(torch.float32)

          # zero the parameter gradients
          optimizer.zero_grad()

          # forward pass and backward pass
          output = model(images)
          loss = criterion(prices, output)
          loss.backward()
          optimizer.step()

          running_loss += loss.item() * images.size(0)
          # ps = torch.exp(output)
          # train_acc += (ps.max(dim=1)[1] == labels.data).type(torch.FloatTensor).mean()

#            if steps % print_every == 0:
      # Turn off gradients for validation, saves memory and computations
      with torch.no_grad():
          valid_loss = validation(model, validloader, criterion, device)
#           scheduler.step(test_loss)

      # if test_accuracy > best_acc:
      if valid_loss < valid_loss_min:
          valid_loss_min = valid_loss
          best_model_wts = copy.deepcopy(model.state_dict())

      print("Epoch: {}/{}... ".format(epoch+1, epochs),
        "Train RMSLE loss: {:.4f}".format(running_loss/len(trainloader.dataset)),
        "Validation RMSLE loss: {:.4f}".format(valid_loss/len(validloader.dataset)),
        )
      # save the losses
      training_loss.append(running_loss/len(trainloader.dataset))
      validation_loss.append(valid_loss/len(validloader.dataset))
      running_loss = 0       
      
  print('Best validation RMSLE loss is {:.4f}'.format(valid_loss_min/len(validloader.dataset)))
  print('Time to complete training {} minutes'.format((time.time() - start) / 60))
  model.load_state_dict(best_model_wts)
  return model, training_loss, validation_loss

In [14]:
def validation(model, validloader, criterion=None, device='cpu'):
    """
    Compute loss on the validation dataset
    :param model - a pretrained model object
    :param validloader - a generator object representing the validataion dataset
    :param criterion - a loss object
    :param device - a string specifying whether to use cuda or cpu
    return a tuple of loss and accuracy
    """
    valid_loss = 0
    model.eval()
    for images, prices in validloader:

        images, prices = images.to(device), prices.to(device)
        output = model(images)
        valid_loss += criterion(prices, output).item() * images.size(0)
    
    return valid_loss

In [15]:
class RMSLELoss(nn.Module):
    r"""Creates a criterion that measures the root mean square error (MAE) between each element in
    the input :math:`x` and target :math:`y`.
    Examples::
        >>> loss = nn.RMSLELoss()
        >>> input = torch.randn(3, 5, requires_grad=True)
        >>> target = torch.randn(3, 5)
        >>> output = loss(input, target)
        >>> output.backward()
    """
    __constants__ = ['reduction']

    def __init__(self, size_average=None, reduce=None, reduction='mean'):
        super(RMSLELoss, self).__init__()

    def forward(self, input, target):
      return rmsle(input, target)

def rmsle(actual_prices, predicted_prices):
    """
    Compute Root Mean Square Log Error
    :param actual_prices - a Tensor vector of log transformed house prices
    :param predicted_prices - a Tensor vector of predicted house prices
    return - Mean Absolute Percentage Error
    """
    return torch.sqrt(torch.mean((actual_prices - predicted_prices) ** 2))
    

class LogCoshLoss(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, actual_prices, predicted_prices):
        ey_t = predicted_prices - actual_prices
        return torch.mean(torch.log(torch.cosh(ey_t + 1e-12)))

# class HuberLoss(nn.Module):
#     r"""Creates a criterion that measures the huber between each element in
#     the input :math:`x` and target :math:`y`.
#     Examples::
#         >>> loss = nn.RMSLELoss()
#         >>> input = torch.randn(3, 5, requires_grad=True)
#         >>> target = torch.randn(3, 5)
#         >>> output = loss(input, target)
#         >>> output.backward()
#     """
#     __constants__ = ['reduction']

#     def __init__(self, size_average=None, reduce=None, reduction='mean', delta=0.1):
#         super(HuberLoss, self).__init__()
#         self.delta = delta

#     def forward(self, input, target):
#       return huber(input, target, self.delta)

# def huber(actual_prices, predicted_prices, delta):
#     loss = np.where(np.abs(actual_prices-predicted_prices) < delta , 0.5*((actual_prices-predicted_prices)**2), delta*np.abs(actual_prices - predicted_prices) - 0.5*(delta**2))
#     return torch.sum(loss)



def optimizer(model, lr=0.001, weight_decay=1e-3/200):
    """
    Define the optimizer used to reduce the loss
    :param model - a pretrained model object
    :param lr - a floating point value defining the learning rate
    :param weight_decay - apply L2 regularization
    return an optimizer object
    """
    if model.__dict__['_modules'].get('fc', None):
        return optim.Adam(model.fc.parameters(), lr=lr, weight_decay=weight_decay)
    return optim.Adam(model.classifier.parameters(), lr=lr, weight_decay=weight_decay)

In [16]:
def check_rmsle_on_test(model, testloader, criterion=None, device='cpu', scale_factor=1e6):  
    """
    Compute the Root Mean Square Error on the test dataset
    :param model - a pretrained model object
    :param testloader - a generator object representing the test dataset
    :param criterion - loss object
    :param device - a string specifying whether to use cuda or cpu
    :param scale_factor - a float to scale the house prices
    return - RMSLE of predicted house prices
    """
    loss = 0
    model.eval()
    with torch.no_grad():
        for data in testloader:
            images, prices = data
            images, prices = images.to(device), prices.to(device)
            outputs = model(images)
            loss += criterion(prices, outputs)
 #           mae += 100 * torch.abs((prices - outputs) / prices)
 #           mae += mape(prices, outputs)
        loss = np.squeeze(loss.numpy()) if device == 'cpu' else np.squeeze(loss.cpu().numpy())

    return loss / len(testloader)

In [17]:
def predict_prices(model, testloader, criterion=None, device='cpu', scale_factor=1e6):  
    """
    Compute the Root Mean Square Error on the test dataset
    :param model - a pretrained model object
    :param testloader - a generator object representing the test dataset
    :param criterion - loss object
    :param device - a string specifying whether to use cuda or cpu
    :param scale_factor - a float to scale the house prices
    return - RMSLE of predicted house prices
    """
    outputs = []
    pricess = []
    loss = 0
    model.eval()
    with torch.no_grad():
        for data in testloader:
            images, prices = data
            images, prices = images.to(device), prices.to(device)
            output = model(images)
            outputs.append(output)
            pricess.append(prices)
            #output = model(images)            
            #loss += criterion(prices, outputs)
 #           mae += 100 * torch.abs((prices - outputs) / prices)
 #           mae += mape(prices, outputs)
        #loss = np.squeeze(loss.numpy()) if device == 'cpu' else np.squeeze(loss.cpu().numpy())


    return outputs, pricess

In [23]:
def plot_losses(training_loss, validation_loss, epochs, split):

  os.makedirs("plots", exist_ok = True)
  plt.figure()
  plt.plot(training_loss, label='training loss')
  plt.plot(validation_loss, label='validation loss')
  plt.legend(frameon=False)
  plt.xlabel('epoch')
  plt.ylabel('loss')
  plt.title('Loss vs. No. of epochs')  
  plt.savefig("plots/logcosh_losses_resnet_"+split+".png")
  plt.show()

##Main

In [18]:
device = "cpu"
gpu = True
if gpu:
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  print('Use device:', device)

Use device: cuda


In [19]:
def train(dataloader, split, epochs, criterion):
  arch = 'resnet50'
  hidden_units = 64
  lr = 0.0001
  #epochs = 50
  
  num_classes = 1 # for regression

  model = build_model(arch, num_classes, hidden_units)

  optim_ = optimizer(model, lr)
  #exp_lr_scheduler = lr_scheduler.StepLR(optim_, step_size=5, gamma=0.1)
  exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optim_, T_max=epochs)
  # default
  # device = "cpu"
  # if gpu:
  #     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  # print('Use device:', device)
  model = model.to(device) 
  print('Training begins...')
  model, training_loss, validation_loss = train_model(model, dataloader['train'], 
                                                      dataloader['valid'], criterion, 
                      optim_, exp_lr_scheduler, epochs, False, device) 
  

  plot_losses(training_loss, validation_loss, epochs, split)

  return model

#model = train(dataloader)

In [20]:
def save_outputs(outputs, prices, split):
  outputs = torch.FloatTensor(outputs)
  outputs = outputs.numpy()
  print(outputs)

  prices = torch.FloatTensor(prices)
  prices = prices.numpy()
  print(prices)

  os.makedirs("model_result", exist_ok=True)

  df_outputs = pd.DataFrame(outputs)
  df_outputs.to_csv('model_result/logcosh_predicted_'+split+'.csv', index=False)
  df_prices =  pd.DataFrame(prices)
  df_prices.to_csv('model_result/logcosh_truevalue_try'+split+'.csv', index=False)


# os.makedirs("plots", exist_ok = True)

# plt.figure()
# plt.plot(outputs, label='prediction')
# plt.plot(a, label='true value')
# plt.legend(frameon=False)
# #plt.xlabel('epoch')
# plt.ylabel('value')
# plt.title('Loss vs. No. of epochs')  
# plt.savefig("plots/values_comparison_2.png")
# plt.show()

# pb = b.numpy()
# px = pd.DataFrame(px)

In [21]:
def plot_predicted_true_value(outputs, prices, split):
  # data to plot
  n_groups = len(outputs)

  # create plot
  fig, ax = plt.subplots()
  index = np.arange(n_groups)
  bar_width = 0.35
  opacity = 0.8

  rects1 = plt.bar(index, outputs, bar_width,
  alpha=opacity,
  color='b',
  label='predicted')

  rects2 = plt.bar(index + bar_width, prices, bar_width,
  alpha=opacity,
  color='g',
  label='true value')

  plt.xlabel('Type')
  plt.ylabel('Value')
  plt.title('Predicted vs True Value')
  #plt.xticks(index + bar_width, ('A', 'B', 'C', 'D', 'E'))
  plt.legend()

  plt.tight_layout()

  plt.savefig("plots/logcosh_value_"+split+".png")
  plt.show()

In [ ]:
def load_dataset():
  image_dir = r'renamed_labeled_dataset'
  train_ids, test_ids, valid_ids = split_into_train_test_valid(image_dir)
  
  for i in range(len(train_ids)):
    print("Split" + str(i))
    make_dirs_copy_images(image_dir, "split"+str(i), train_ids[i], test_ids[i], valid_ids[i])
    df = parse_gauze_info(image_dir, 'labels_4x4.csv')

    train_batch_size= 32
    test_batch_size = 1
    valid_batch_size = 32
    img_size = 128
    transform_img_size = 224

    #criterion = torch.nn.MSELoss()
    #criterion = RMSLELoss()
    #criterion = torch.nn.L1Loss()
    #criterion = torch.nn.SmoothL1Loss()
    criterion = LogCoshLoss()

    image_datasets = {}
    dataloader = {}
    image_transforms = transform_images(size=transform_img_size)
    try:
      for data_type, gauze_ids, batch_size, shuffle in zip(['train', 'test', 'valid'],
                          [train_ids[i], test_ids[i], valid_ids[i]],
                          [train_batch_size, test_batch_size, valid_batch_size],
                          [True, False, True]):
        image_datasets.update(load_data(image_dir+"/"+"split"+str(i), data_type, df, gauze_ids, img_size, image_transforms[data_type]))
        dataloader.update(data_loader(image_datasets[data_type], data_type, batch_size, shuffle))
    except:  
        raise Exception('Loading of dataset failed...')

    model = train(dataloader, "split"+str(i), 30, criterion)

    test_loss = check_rmsle_on_test(model, dataloader['test'], criterion, device)
    print('Test set RMSLE loss is {:.4f}'.format(test_loss))

    outputs, prices = predict_prices(model, dataloader['test'], criterion,
                  device='cuda', scale_factor=1)
    
    save_outputs(outputs, prices, "split"+str(i))
    plot_predicted_true_value(outputs, prices, "split"+str(i))

    
load_dataset()

Split0
Training begins...


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch: 1/30...  Train RMSLE loss: 3.8893 Validation RMSLE loss: 3.6330
Epoch: 2/30...  Train RMSLE loss: 3.6498 Validation RMSLE loss: 3.4307
Epoch: 3/30...  Train RMSLE loss: 3.4295 Validation RMSLE loss: 3.2393
Epoch: 4/30...  Train RMSLE loss: 3.2124 Validation RMSLE loss: 3.0830
Epoch: 5/30...  Train RMSLE loss: 3.0044 Validation RMSLE loss: 2.9685
Epoch: 6/30...  Train RMSLE loss: 2.8128 Validation RMSLE loss: 2.8093
Epoch: 7/30...  Train RMSLE loss: 2.6565 Validation RMSLE loss: 2.6873
Epoch: 8/30...  Train RMSLE loss: 2.5158 Validation RMSLE loss: 2.5548
Epoch: 9/30...  Train RMSLE loss: 2.3738 Validation RMSLE loss: 2.4084
Epoch: 10/30...  Train RMSLE loss: 2.2709 Validation RMSLE loss: 2.2601
Epoch: 11/30...  Train RMSLE loss: 2.1817 Validation RMSLE loss: 2.1363
Epoch: 12/30...  Train RMSLE loss: 2.0792 Validation RMSLE loss: 2.0341
Epoch: 13/30...  Train RMSLE loss: 2.0114 Validation RMSLE loss: 1.9613
Epoch: 14/30...  Train RMSLE loss: 1.9852 Validation RMSLE loss: 1.9090
E